# Evaluate the box below to initialize the web UI.

In [2]:
%%html
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

<button id="do_run_all">Click to run all</button>
<script>
$("#do_run_all").click(
    function () {
        $("#run_all_cells").click();
    }
);
</script>

In [3]:
import ipywidgets as widgets
import os
import pandas as pd
import pprint
import qgrid
import ray
import subprocess
import sys
import tempfile
import time
import math

from IPython.display import display

ray.init(redis_address=os.environ["REDIS_ADDRESS"])

{'local_scheduler_socket_names': ['/tmp/scheduler44634541'],
 'node_ip_address': '127.0.0.1',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store94210907', manager_name='/tmp/plasma_manager53461663', manager_port=18713)],
 'redis_address': '127.0.0.1:63364'}

**Object search.**

In [4]:
object_search = widgets.Text(
    value="",
    placeholder="Object ID",
    description="Search for an object:",
    disabled=False
)
display(object_search)

def handle_submit(sender):
    pp = pprint.PrettyPrinter()
    pp.pprint(ray.global_state.object_table(object_search.value))

object_search.on_submit(handle_submit)

**Task search.**

In [5]:
task_search = widgets.Text(
    value="",
    placeholder="Task ID",
    description="Search for a task:",
    disabled=False
)
display(task_search)

def handle_submit(sender):
    pp = pprint.PrettyPrinter()
    pp.pprint(ray.global_state.task_table(task_search.value))

task_search.on_submit(handle_submit)

#### Task Time Series.

In [6]:
def time_series():
    start = 0
    end = time.time()
    
    granularity = 1
    earliest = end
    latest = 0
    tasks = ray.global_state.task_profiles(start=start, end=end)
    for task_id, data in tasks.items():
        if data["score"] > latest:
            latest = data["score"]
        if data["score"] < earliest:
            earliest = data["score"]
    num_buckets = math.ceil((latest - earliest) / granularity)
    
    buckets = []
    
    for i in range(0, num_buckets, granularity):
        start = i * granularity + earliest
        end = ((i + 1) * granularity) + earliest
        t = ray.global_state.task_profiles(start=math.floor(start), end=math.ceil(end))
        buckets.append(len(t))
        
    start_point = earliest
    end_point = start_point + granularity * num_buckets
    
    return buckets, start_point, end_point, granularity

import numpy as np
import scipy.special
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.resources import CDN
from bokeh.io import output_notebook
output_notebook(resources=CDN)
from bokeh.models import Range1d
import math


buckets, earliest, latest, gran = time_series()
if len(buckets) is not 0: 
    desired_range = (0, max(buckets)+10)
    distr = []
    for x in range(len(buckets)):
        for y in range(buckets[x]):
            distr.append(x)

    p = figure(title="Task Time Series",tools=["save", "hover", "wheel_zoom", "box_zoom", "pan"],
                background_fill_color="#FFFFFF", y_range = desired_range, x_range = (0, latest - earliest))
    bins = [(i - 1) * gran for i in range(len(buckets) + 2)]
    hist, bin_edges = np.histogram(distr, bins=bins)

    p.quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:],
            fill_color="#B3B3B3", line_color="#033649")

    p.xaxis.axis_label = 'Time in seconds'
    p.yaxis.axis_label = 'Number of concurrent tasks'

    show(gridplot(p, ncols=1, plot_width=500, plot_height=500, toolbar_location="below"))

Loading BokehJS ...

#### Cluster Usage. 

In [7]:
import time
import math
def heat_map():
    start = 0
    end = time.time()
   
    granularity = 1
    earliest = end
    latest = 0
    tasks = ray.global_state.task_profiles(start=start, end=end)
    for task_id, data in tasks.items():
        if data["score"] > latest:
            latest = data["score"]
        if data["score"] < earliest:
            earliest = data["score"]
    num_buckets = math.ceil((latest - earliest) / granularity)
    buckets = [0] * num_buckets
    
    worker_info = ray.global_state.workers()
    num_tasks = []
    nodes = []
    times = []
    start_point = earliest
    end_point = len(buckets) * granularity + earliest
    
    for i in range(0, len(buckets), granularity):
        start = i * granularity + earliest
        end = (i + 1) * granularity + earliest
        t = ray.global_state.task_profiles(start=math.floor(start), end=math.ceil(end))
        
        node_to_num = dict()
        for task_id, data in t.items():
            worker = data["worker_id"]
            node = worker_info[worker]["node_ip_address"]
            if node not in node_to_num:
                node_to_num[node] = 0
            node_to_num[node] += 1
            
        for node_ip, counter in node_to_num.items():
            num_tasks.append(node_to_num[node_ip])
            nodes.append(node_ip)
            times.append(i)
            
    return nodes, times, num_tasks
 
from math import pi
import pandas as pd
import random
import numpy as np
from bokeh.io import show, output_notebook
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar,
)
from bokeh.plotting import figure

output_notebook()
 
node_ip_address, times, num_tasks = heat_map()

if len(node_ip_address) is not 0: 
 
    df = pd.DataFrame({"node_ip_address":node_ip_address, "time":times, "num_tasks":num_tasks})

    colors = ["#FFFFFF", "#E8E8E8", "#DCDCDC", "#D3D3D3", "#B8B8B8", "#A8A8A8", "#696969", "#383838", "#000000"]
    mapper = LinearColorMapper(palette=colors, low=df.num_tasks.min() - 1, high=df.num_tasks.max() + 1)
    source = ColumnDataSource(df)

    TOOLS = "hover,save,xpan,box_zoom,reset,xwheel_zoom"

    p = figure(title="Cluster Usage", y_range=list(set(node_ip_address)),
               x_axis_location="above", plot_width=900, plot_height=500,
               tools=TOOLS, toolbar_location='below')

    p.grid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "10pt"
    p.axis.major_label_standoff = 0
    p.xaxis.major_label_orientation = pi / 3

    p.rect(x="time", y="node_ip_address", width=1, height=1,
           source=source,
           fill_color={'field': 'num_tasks', 'transform': mapper},
           line_color=None)

    color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="8pt",
                         ticker=BasicTicker(desired_num_ticks=len(colors)),
                         label_standoff=6, border_line_color=None, location=(0, 0))
    p.add_layout(color_bar, 'right')

    p.select_one(HoverTool).tooltips = [
         ('Node IP Address', '@node_ip_address'),
         ('Number of tasks running', '@num_tasks'),
         ('Time', '@time')
    ]


    p.xaxis.axis_label = "Time in seconds"
    p.yaxis.axis_label = "Node IP Address"

    show(p)

Loading BokehJS ...

#### Task Completion Time Distribution.

In [8]:
import time
from bokeh.models import Range1d
tasks = ray.global_state.task_profiles(start=0, end=time.time())
if len(tasks) is not 0: 
    unique = set()
    distr = []
    mu, sigma = 0, 0.5

    for task_id, data in tasks.items():
        unique.add(data["store_outputs_end"] - data["get_task_start"])
        distr.append(data["store_outputs_end"] - data["get_task_start"])

    hist, bin_edges = np.histogram(distr, bins = range(len(unique)))
    p = figure(title="Task Completion Time Distribution",tools=["save", "hover", "wheel_zoom", "box_zoom", "pan"],
                background_fill_color="#FFFFFF", x_range = (0,max(distr) +2), y_range = (0, max(hist)+2))
    p.quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:],
            fill_color="#B3B3B3", line_color="#033649")
    x = np.linspace(-2, 2, 1000)

    p.xaxis.axis_label = 'Time in seconds'
    p.yaxis.axis_label = 'Number of concurrent tasks'

    show(gridplot(p, ncols=1, plot_width=500, plot_height=500, toolbar_location="below"))